# Studying the orbit of satellite galaxies

The purpose of this tutorial is to help *you* with modelling and studying the orbit of galaxies.
There are a few things you need to consider when creating your model.
- Firstly: what data are you using? When you start your model, you need to have some data on the current 6D properties of your galaxy. The two main methods (that may either produce the same or different results) are 1. manually inputting the 6D data as a list, or 2. using the orbit.Orbit.from_name method. The from_name method imports the data from the SIMBAD database automatically.
- Next: what potential are you using for the Milky Way? the galpy package's default potential is 'MWPotential2014', from *Include the source*, however it also includes others like McMillan17, or "". You can also instantiate the potential from scratch, using different potential objects like "".
- And: Do you want to include the LMC potential? In recent studies *cite?*, it's been found that the LMC's effects on the Milky Way are fairly substantial, and depending on the distance from the Milky Way's centre, it's inclusion will affect how satellite galaxies orbit the Milky Way. You can either include it by making a simplified potential, ..... Or, you can include its affects on the acceleration of the Milky Way galactic centre, by calculating the resulting force of the LMC on the MW.
- Lastly: For what timeframe do you want to model your orbit? You can integrate your orbit forwards, backwards, or even both to compare the past modelled orbit to the predicted future orbit. You can also choose exactly how far back (or far forwards) you want to integrate the orbit, just keep in mind that the further you model from the present, the less accurate the model may become. Most of the time, integrating your galaxy enough for one full orbit is sufficient.

In [1]:
# Importing our libraries and packages required for the orbit
from galpy.orbit import Orbit
from galpy import potential
import numpy as np
import matplotlib.pyplot as plt
from galpy.util import conversion

# Importing potentials.
from galpy.potential import MWPotential2014,ChandrasekharDynamicalFrictionForce,HernquistPotential,MovingObjectPotential

## Instantiating our galaxy
The first thing that we do is create a galpy Orbit object that represents our satellite galaxy. If you're inputting the initial conditions by hand, you want to make sure that you're inputting it properly so that the orbit knows what values you're using. You can set it up with [R,vR,vT,z,vz,phi] coordinates very easily; **Do a note on physical units?**
```python  
o = Orbit([R,vR,vT,z,vz,phi])
```
If you have [RA(deg),DEC(deg),Dist(kpc),$\mu_\alpha^*$(mas/yr), $\mu_\delta$(mas/yr),$v_r$(km/s)], you don't need to convert your data into other coordinates, and can instantiate simply;
```python
o = Orbit([RA,DEC,Dist,pm_ra,pm_dec,vr], radec=True)
```
Lastly, you can also instantiate the orbit with retrieved 6D conditions from SIMBAD. Keep in mind that the name query needs to be identical to how it is listed on the SIMBAD database;
```python
name = "my galaxy's name"
o = Orbit.from_name(name)
```

Sometimes, you may want to use physical coordinates immediately (.. write about ro and vo).
Or, you may want to use a different solar motion than the default. The default solar motion is from Schoenrich et al. (2010), where the solar motion [U,V,W] = [11.1,12.24,7.25].

Also, if you need outputs to be in the natural galpy coordinates, you can include `o.turn_physical_off()` in your script.

## Timescale choice
Now that the orbit has been initialized,our next step is to choose our timescale. This is not too important immediately, and can always be adjusted later. We choose how far we integrate, for example `t = 2` for 2Gyr, then we convert it into galpy units by introducing a conversion factor `to = conversion.time_in_Gyr(ro=ro,vo=vo)`. We also include how many time steps we'd like to integrate with `n=1001` Lastly we decide whether we are integrating our orbit forwards or backwards, by simply defining our time array to increase from 0 (forwards) `ts= np.linspace(0.,t/to,n)`, or to decrease from 0 (backwards) `ts= np.linspace(0.,-t/to,n)`.
This can be done using other methods, but the important thing is that we create an array that spans a timeframe including the initial time (current time, defined as t=0).

## Deciding on the potential
This is a very important step that will impact the result of the orbit model the most.
As mentioned previously, the first step is to decide what 'base' potential we are going to use for the Milky Way. Depending on the desired model, you can choose which Milky Way potential suits your needs. Check [galpy potentials doc](https://docs.galpy.org/en/latest/reference/potential.html) for more information on this.
If you don't specify a potential, the default will be MWPotential2014, which represents the sum of the halo and disk (but not the centre) **double check w/ citation**.

### Are you including the LMC?
Including the effect of the LMC on the Milky Way introduces some irregular but realistic motions.